<a href="https://colab.research.google.com/github/brendanpshea/colab-utilities/blob/main/SQL_Python_Quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import sqlite3
import pandas as pd
import requests
import tempfile
import json
from IPython.display import display, HTML, clear_output
from ipywidgets import (Textarea, Button, VBox, HBox, Layout, IntProgress,
                        Tab, IntText, Label, Box, HTML as HTMLWidget)

def get_table_schemas(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    return [(table[0], cursor.execute(f"PRAGMA table_info({table[0]})").fetchall())
            for table in cursor.fetchall()]

def render_table_schemas(schemas):
    schema_html = "<h3>Database Schema:</h3><ul>"
    for table_name, columns in schemas:
        column_info = ", ".join(f"{column[1]} {column[2]}" for column in columns)
        schema_html += f"<li><b>{table_name}</b> ({column_info})</li>"
    schema_html += "</ul>"
    return schema_html

def execute_query(conn, query):
    try:
        return pd.read_sql_query(query, conn)
    except Exception as e:
        raise ValueError(f"Error executing query: {str(e)}")

class SQLQuiz:
    def __init__(self, db_path, questions, answers):
        self.conn = sqlite3.connect(db_path)
        self.questions = questions
        self.answers = answers
        self.current_index = 0
        self.setup_ui()

    def setup_ui(self):
        self.progress_bar = IntProgress(min=0, max=len(self.questions), value=1, description='Progress:')
        self.question_label = HTMLWidget(value='')

        self.text_area = Textarea(placeholder='Type your SQL query here...',
                                  layout=Layout(width='100%', height='150px'))
        self.submit_button = Button(description="Submit", button_style='success')
        self.clear_button = Button(description="Clear", button_style='warning')
        self.hint_button = Button(description="Hint", button_style='info')

        self.skip_input = IntText(value=1, min=1, max=len(self.questions), layout=Layout(width='60px'))
        self.skip_button = Button(description="Skip to", button_style='info')
        self.skip_box = HBox([Label('Go to question:'), self.skip_input, self.skip_button])

        self.submit_button.on_click(self.submit_query)
        self.clear_button.on_click(self.clear_query)
        self.skip_button.on_click(self.skip_to_question)
        self.hint_button.on_click(self.show_hint)

        self.query_box = VBox([self.text_area,
                               HBox([self.submit_button, self.clear_button, self.hint_button]),
                               self.skip_box])

        self.results_area = HTMLWidget(value='')
        self.try_again_button = Button(description="Try Again", button_style='warning')
        self.next_button = Button(description="Next Question", button_style='info')
        self.try_again_button.on_click(self.try_again)
        self.next_button.on_click(self.next_question)

        self.results_box = VBox([self.results_area, HBox([self.try_again_button, self.next_button])])

        self.tab = Tab(children=[self.query_box, self.results_box])
        self.tab.set_title(0, 'Query')
        self.tab.set_title(1, 'Results')

        self.main_box = VBox([self.progress_bar, self.question_label, self.tab])

    def display_current_question(self, _=None):
        clear_output(wait=True)
        schemas = get_table_schemas(self.conn)
        self.question_label.value = (f"<h3>Question {self.current_index + 1} of {len(self.questions)}:</h3>"
                                     f"<p>{self.questions[self.current_index]}</p>"
                                     f"{render_table_schemas(schemas)}")

        self.text_area.value = ''
        self.submit_button.disabled = False
        self.skip_input.max = len(self.questions)
        self.progress_bar.value = self.current_index + 1
        self.tab.selected_index = 0  # Switch to Query tab
        display(self.main_box)

    def submit_query(self, _):
        user_query = self.text_area.value.strip()
        if not user_query.lower().startswith('select'):
            self.display_error("Please enter a valid SELECT query.")
            return

        try:
            user_result = execute_query(self.conn, user_query)
            correct_result = execute_query(self.conn, self.answers[self.current_index])
            self.display_results(user_result, correct_result)
        except ValueError as e:
            self.display_error(str(e))

    def display_results(self, user_result, correct_result):
        user_rows, user_cols = user_result.shape
        correct_rows, correct_cols = correct_result.shape
        count_info = (f"<p>Your query yielded {user_rows} rows and {user_cols} columns. "
                      f"The expected result had {correct_rows} rows and {correct_cols} columns.</p>")

        if user_result.equals(correct_result):
            feedback = "<h3 style='color: green;'>Correct! Your query produced the expected result.</h3>"
            self.next_button.disabled = False
            self.try_again_button.disabled = True
        else:
            feedback = "<h3 style='color: red;'>Incorrect. Please try again.</h3>"
            self.next_button.disabled = True
            self.try_again_button.disabled = False

        results_html = feedback + count_info
        results_html += "<h4>Your Results (first five rows):</h4>"
        results_html += user_result.head().to_html()
        results_html += "<h4>Expected Results (first five rows):</h4>"
        results_html += correct_result.head().to_html()

        self.results_area.value = results_html
        self.tab.selected_index = 1  # Switch to Results tab

    def display_error(self, message):
        self.results_area.value = f"<h3 style='color: red;'>Error: {message}</h3>"
        self.tab.selected_index = 1  # Switch to Results tab

    def clear_query(self, _):
        self.text_area.value = ''

    def try_again(self, _):
        self.tab.selected_index = 0  # Switch back to Query tab

    def next_question(self, _):
        self.current_index += 1
        if self.current_index < len(self.questions):
            self.display_current_question()
        else:
            self.main_box.children = [HTMLWidget(value="<h2>All questions completed. Well done!</h2>")]
            display(self.main_box)

    def skip_to_question(self, _):
        new_index = self.skip_input.value - 1
        if 0 <= new_index < len(self.questions):
            self.current_index = new_index
            self.display_current_question()
        else:
            self.display_error(f"Invalid question number. Please enter a number between 1 and {len(self.questions)}.")

    def show_hint(self, _):
        correct_query = self.answers[self.current_index]
        words = correct_query.split()
        hint = []
        for i, word in enumerate(words):
            if i % 2 == 0:
                hint.append(word)
            else:
                hint.append('____' * (len(word) // 4 + 1))  # Adjust underscores based on word length
        hint = " ".join(hint)
        self.results_area.value = f"<h3>Hint:</h3><p>{hint}</p>"
        self.tab.selected_index = 1  # Switch to Results tab
def sql_select_quiz_from_id(quiz_id="books"):
    if quiz_id == "books":
        db_url = "https://github.com/brendanpshea/database_sql/raw/main/data/sci_fi_books.db"
        json_url = "https://github.com/brendanpshea/database_sql/raw/main/quiz/sql_book_quiz.json"
    sql_select_quiz_url(db_url, json_url)

def sql_select_quiz_url(db_url, json_url):
    with tempfile.NamedTemporaryFile(delete=False) as temp_db:
        db_path = temp_db.name
        response = requests.get(db_url)
        temp_db.write(response.content)

    response = requests.get(json_url)
    quiz_data = json.loads(response.text)

    questions = [item['question'] for item in quiz_data]
    answers = [item['answer'] for item in quiz_data]

    quiz = SQLQuiz(db_path, questions, answers)
    quiz.display_current_question()

In [14]:
sql_select_quiz_from_id("books")

In [18]:
import inspect
import io
import sys
from IPython.display import display, HTML, clear_output
from ipywidgets import (Textarea, Button, VBox, HBox, Layout, IntProgress,
                        Tab, IntText, Label, Box, HTML as HTMLWidget)

# Comprehensive test inputs
TEST_INPUTS = {
    'int': [
        -1000000,  # Large negative
        -1,        # Negative one
        0,         # Zero
        1,         # Positive one
        1000000,   # Large positive
        2**31 - 1, # Max 32-bit integer
        -(2**31),  # Min 32-bit integer
        42,        # Common test number
        -273,      # Negative edge case (absolute zero in Celsius)
        99         # Two-digit number
    ],
    'str': [
        "",            # Empty string
        "a",           # Single character
        "hello world", # Common test string with space
        "Python3.9!",  # String with numbers and special characters
        "   ",         # String with only spaces
        "A" * 1000,    # Long string with repeated character
        "aBcDeF",      # Mixed case
        "\n\t\r",      # String with escape characters
        "😊",          # Unicode character (emoji)
        "The quick brown fox jumps over the lazy dog"  # Pangram
    ]
}

class PythonQuiz:
    def __init__(self, questions):
        self.questions = questions
        self.current_index = 0
        self.setup_ui()

    def setup_ui(self):
        self.progress_bar = IntProgress(min=0, max=len(self.questions), value=1, description='Progress:')
        self.question_label = HTMLWidget(value='')

        self.code_area = Textarea(placeholder='Write your Python function here...',
                                  layout=Layout(width='100%', height='200px'))
        self.run_button = Button(description="Run", button_style='success')
        self.clear_button = Button(description="Clear", button_style='warning')

        self.skip_input = IntText(value=1, min=1, max=len(self.questions), layout=Layout(width='60px'))
        self.skip_button = Button(description="Skip to", button_style='info')
        self.skip_box = HBox([Label('Go to question:'), self.skip_input, self.skip_button])

        self.run_button.on_click(self.run_code)
        self.clear_button.on_click(self.clear_code)
        self.skip_button.on_click(self.skip_to_question)

        self.code_box = VBox([self.code_area,
                              HBox([self.run_button, self.clear_button]),
                              self.skip_box])

        self.results_area = HTMLWidget(value='')
        self.try_again_button = Button(description="Try Again", button_style='warning')
        self.next_button = Button(description="Next Question", button_style='info')
        self.try_again_button.on_click(self.try_again)
        self.next_button.on_click(self.next_question)

        self.results_box = VBox([self.results_area, HBox([self.try_again_button, self.next_button])])

        self.tab = Tab(children=[self.code_box, self.results_box])
        self.tab.set_title(0, 'Code')
        self.tab.set_title(1, 'Results')

        self.main_box = VBox([self.progress_bar, self.question_label, self.tab])

    def display_current_question(self):
        clear_output(wait=True)
        question = self.questions[self.current_index]

        # Execute correct answer
        exec(question['answer'])
        correct_function = locals()[question['function_name']]

        # Generate sample input/outputs
        sample_inputs = TEST_INPUTS[question['input_type']][:3]
        sample_outputs = [correct_function(inp) for inp in sample_inputs]
        samples = [f"Input: {inp}, Output: {out}" for inp, out in zip(sample_inputs, sample_outputs)]

        self.question_label.value = (f"<h3>Question {self.current_index + 1} of {len(self.questions)}:</h3>"
                                     f"<p>{question['description']}</p>"
                                     f"<p>Function signature: <code>{question['function_signature']}</code></p>"
                                     f"<p>Sample input/outputs:</p>"
                                     f"<ul>{''.join([f'<li>{sample}</li>' for sample in samples])}</ul>")

        self.code_area.value = f"def {question['function_signature']}:\n    # Your code here\n    pass"
        self.run_button.disabled = False
        self.skip_input.max = len(self.questions)
        self.progress_bar.value = self.current_index + 1
        self.tab.selected_index = 0  # Switch to Code tab
        display(self.main_box)

    def run_code(self, _):
        question = self.questions[self.current_index]
        user_code = self.code_area.value

        # Capture stdout
        old_stdout = sys.stdout
        redirected_output = sys.stdout = io.StringIO()

        try:
            # Execute user's code
            exec(user_code)
            user_function = locals()[question['function_name']]

            # Execute correct answer
            exec(question['answer'])
            correct_function = locals()[question['function_name']]

            # Run tests
            inputs = TEST_INPUTS[question['input_type']]
            all_correct = True
            results = []
            for test_input in inputs:
                user_output = user_function(test_input)
                expected_output = correct_function(test_input)
                is_correct = user_output == expected_output
                all_correct &= is_correct
                results.append({
                    'input': test_input,
                    'expected': expected_output,
                    'actual': user_output,
                    'is_correct': is_correct
                })

            # Generate results HTML
            results_html = self.generate_results_html(results, all_correct)

            # Display results
            self.results_area.value = results_html
            self.tab.selected_index = 1  # Switch to Results tab

            # Enable/disable buttons
            self.next_button.disabled = not all_correct
            self.try_again_button.disabled = all_correct

        except Exception as e:
            self.results_area.value = f"<h3 style='color: red;'>Error:</h3><pre>{str(e)}</pre>"
            self.tab.selected_index = 1  # Switch to Results tab
        finally:
            sys.stdout = old_stdout
            captured_output = redirected_output.getvalue()
            if captured_output:
                self.results_area.value += f"<h4>Printed Output:</h4><pre>{captured_output}</pre>"

    def generate_results_html(self, results, all_correct):
        html = "<h3 style='color: green;'>All tests passed!</h3>" if all_correct else "<h3 style='color: red;'>Some tests failed. Please try again.</h3>"
        html += "<table border='1'><tr><th>Input</th><th>Expected Output</th><th>Your Output</th><th>Result</th></tr>"
        for result in results:
            color = 'green' if result['is_correct'] else 'red'
            html += f"<tr style='color: {color};'>"
            html += f"<td>{result['input']}</td>"
            html += f"<td>{result['expected']}</td>"
            html += f"<td>{result['actual']}</td>"
            html += f"<td>{'Correct' if result['is_correct'] else 'Incorrect'}</td>"
            html += "</tr>"
        html += "</table>"
        return html

    def clear_code(self, _):
        question = self.questions[self.current_index]
        self.code_area.value = f"def {question['function_signature']}:\n    # Your code here\n    pass"

    def try_again(self, _):
        self.tab.selected_index = 0  # Switch back to Code tab

    def next_question(self, _):
        self.current_index += 1
        if self.current_index < len(self.questions):
            self.display_current_question()
        else:
            self.main_box.children = [HTMLWidget(value="<h2>All questions completed. Well done!</h2>")]
            display(self.main_box)

    def skip_to_question(self, _):
        new_index = self.skip_input.value - 1
        if 0 <= new_index < len(self.questions):
            self.current_index = new_index
            self.display_current_question()
        else:
            self.results_area.value = f"<h3 style='color: red;'>Error: Invalid question number. Please enter a number between 1 and {len(self.questions)}.</h3>"
            self.tab.selected_index = 1  # Switch to Results tab
# Example questions
questions = [
    {
        'description': 'Write a function that takes an integer and returns its absolute value.',
        'function_signature': 'absolute_value(n)',
        'function_name': 'absolute_value',
        'input_type': 'int',
        'answer': 'def absolute_value(n):\n    return abs(n)'
    },
    {
        'description': 'Write a function that takes a string and returns its length.',
        'function_signature': 'string_length(s)',
        'function_name': 'string_length',
        'input_type': 'str',
        'answer': 'def string_length(s):\n    return len(s)'
    }
]

quiz = PythonQuiz(questions)
quiz.display_current_question()